# M3DB NoteBook

## I/ Installation et configuration

M3 est développé en GO.

### a) Installation de go
Télécharger l'archive de package go selon l'OS dans la [page officel](https://golang.org/dl/).
Désarchiver le package avec le command suivant (modifier le chemin désiré et le nom de l'archive si nécessaire): 
```bash
$ tar -C $HOME/local -xzf go$VERSION.$OS-$ARCH.tar.gz
```

Ajoutez le répertoire `bin` de GO à la variable d'environnement PATH : 

```bash
$ export PATH=$PATH:export PATH=$PATH:$HOME/local/go/bin
```
#### Tester l'installation de GO
On peut utiliser les fichiers de test officiels dans le répertoire `test`.
```bash
> cd test
> go build helloworld.go
> ./helloworld
```
Si tout est bien installé, une ligne s'afficherai dans la console : 
```bash
hello, world
```
### b) Installation du M3DB
D'abord fork le projet de m3db sur github : https://github.com/m3db/m3

Vérifier que la variable GOPATH est bien settée:
```bash
echo $GOPATH
```

Exécuter les commandes suivantes: 
```bash
export working_dir=$GOPATH/src/github.com/m3db
mkdir -p $working_dir
```

```bash
git clone https://github.com/m3db/m3.git

cd $working_dir/m3
make m3dbnode
# to build a local m3dbnode process
make m3dbnode (note that we currently require at least Go 1.10 or higher)

# run it with the sample configuration
./bin/m3dbnode -f ./src/dbnode/config/m3dbnode-local-etcd.yml
```

Si vous n'avez pas les droits root, changer la valeur de 
- filePathPrefix: /var/lib/m3db
- cacheDir: /var/lib/m3kv
dans le fichier configuration, src/dbnode/config/m3dbnode-local-etcd.yml

```bash
fs:
    filePathPrefix: /var/lib/m3db

  config:
      service:
          env: default_env
          zone: embedded
          service: m3db
          cacheDir: /var/lib/m3kv
```

### c) Test installation M3DB

#### Creating a namespace to store metrics

```
curl -X POST http://localhost:7201/api/v1/database/create -d '{
  "type": "local",
  "namespaceName": "default",
  "retentionTime": "2h"
}'
```


#### Write a datapoint

```
curl http://localhost:9003/writetagged -s -X POST -d '{
  "namespace": "default",
  "id": "foo",
  "tags": [
    {
      "name": "__name__",
      "value": "user_login"
    },
    {
      "name": "city",
      "value": "new_york"
    },
    {
      "name": "endpoint",
      "value": "/request"
    }
  ],
  "datapoint": {
    "timestamp":'"$(date +"%s")"',
        "value": 42.123456789
  }
}'
 

#### Query for reverse indexed time series data

```
curl http://localhost:9003/query -s -X POST -d '{
  "namespace": "default",
  "query": {
    "regexp": {
      "field": "city",
      "regexp": ".*"
    }
  },
  "rangeStart": 0,
  "rangeEnd":'"$(date +"%s")"'
}' | jq .
```


results :
```bash
{
  "results": [
    {
      "id": "foo",
      "tags": [
        {
          "name": "__name__",
          "value": "user_login"
        },
        {
          "name": "city",
          "value": "new_york"
        },
        {
          "name": "endpoint",
          "value": "/request"
        }
      ],
      "datapoints": [
        {
          "timestamp": 1588004137,
          "value": 42.123456789
        }
      ]
    }
  ],
  "exhaustive": true
}
```